## Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
class Block(tf.keras.Model):
  def __init__(self, filters, kernel_size, repetitions, pool_size=2, stride=2):
    super(Block, self).__init__()
    self.filters = filters
    self.kernel_size = kernel_size
    self.repetitions = repetitions

    for i in range(repetitions):

      vars(self)[f'conv2D_{i}'] = tf.keras.layers.Conv2D(filters=filters,
                                                         kernel_size=kernel_size,
                                                         activation='relu',
                                                         padding='same')
    self.max_pool = tf.keras.layers.MaxPool2D(pool_size=pool_size,
                                               strides=stride)

  def call(self, inputs, training=False):
    conv2D_0 = self.conv2D_0

    x = conv2D_0(inputs)
    for i in range(1, self.repetitions):
      conv2D_i = vars(self)[f'conv2D_{i}']
      x = conv2D_i(x)

    max_pool = self.max_pool(x)
    return max_pool



In [ ]:
class MyVGG(tf.keras.Model):
  def __init__(self, num_classes, input_shape=(None, None, 3)):
    super(MyVGG, self).__init__()
    self.input_layer = tf.keras.layers.InputLayer(input_shape=input_shape) # Define input layer
    self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
    self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
    self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
    self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
    self.block_e = Block(filters=512, kernel_size=3, repetitions=3)

    self.flatten = tf.keras.layers.Flatten()
    self.fc = tf.keras.layers.Dense(256, activation='relu')
    self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

  def call(self, inputs):
    x = self.block_a(inputs)
    x = self.block_b(x)
    x = self.block_c(x)
    x = self.block_d(x)
    x = self.block_e(x)
    x = self.flatten(x)

    x = self.fc(x)
    x = self.classifier(x)
    return x

## Loading the dataset

In [ ]:
dataset = tfds.load('cats_vs_dogs', split=tfds.Split.TRAIN)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.T3P7MS_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


## Training

In [ ]:
# Initialize VGG with the number of classes
vgg = MyVGG(num_classes=2, input_shape=(224, 224, 3))

# Compile with losses and metrics
vgg.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define preprocessing function
def preprocess(features):
    # Resize and normalize
    image = tf.image.resize(features['image'], (224, 224))
    return tf.cast(image, tf.float32) / 255., features['label']


# Apply transformations to dataset
dataset = dataset.map(preprocess).batch(32)

# Train the custom VGG model
vgg.fit(dataset, epochs=1)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


727/727 ━━━━━━━━━━━━━━━━━━━━ 116s 138ms/step - accuracy: 0.5411 - loss: 0.6890


In [ ]:
vgg.fit(dataset, epochs=10)

Epoch 1/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 88s 121ms/step - accuracy: 0.6371 - loss: 0.6488
Epoch 2/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 88s 122ms/step - accuracy: 0.6635 - loss: 0.6161
Epoch 3/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 142s 121ms/step - accuracy: 0.6796 - loss: 0.5964
Epoch 4/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 88s 121ms/step - accuracy: 0.6916 - loss: 0.5829
Epoch 5/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 88s 122ms/step - accuracy: 0.7007 - loss: 0.5723
Epoch 6/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 142s 121ms/step - accuracy: 0.7076 - loss: 0.5632
Epoch 7/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 88s 121ms/step - accuracy: 0.7146 - loss: 0.5550
Epoch 8/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 88s 122ms/step - accuracy: 0.7206 - loss: 0.5473
Epoch 9/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 142s 121ms/step - accuracy: 0.7275 - loss: 0.5400
Epoch 10/10
727/727 ━━━━━━━━━━━━━━━━━━━━ 142s 121ms/step - accuracy: 0.7307 - loss: 0.5330
